## Install Required Dependencies

In [2]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep polars = {version = "0.28.0", features = ["describe", "lazy", "ndarray"]}
:dep color-eyre = {version = "0.6.2"}
:dep ndarray = {version = "0.15.6"}
:dep smartcore = {version = "0.3.1"}
// or
// :dep polars = { git = "https://github.com/pola-rs/polars"}"}
// :dep polars = { git = "https://github.com/yaahc/color-eyre"}
// :dep polars = { git = "https://github.com/rust-ndarray/ndarray"}

## Import Modules

In [ ]:
use polars::prelude::*;
use polars::frame::DataFrame;
use std::path::Path;
use ndarray::{ArrayBase, DataMut, Dimension, concatenate, Axis};
use plotters::prelude::*;
use smartcore::linalg::basic::matrix::DenseMatrix;
use ndarray::prelude::*;


## Load Dataset

In [ ]:
fn read_data_frame_from_csv(
    csv_file_path: &Path,
) -> DataFrame {
    CsvReader::from_path(csv_file_path)
        .expect("Cannot open file.")
        .has_header(true)
        .finish()
        .unwrap()
}

let iris_file_path: &Path = Path::new("dataset/Iris.csv");
let iris_df: DataFrame = read_data_frame_from_csv(iris_file_path);

## About The Dataset

### Shape

In [ ]:
iris_df.shape()

### Head

In [ ]:
iris_df.head(Some(5))

### Tail

In [ ]:
iris_df.tail(Some(5))

### Describe

In [ ]:
iris_df.describe(None)

### Columns

In [ ]:
let column_names = iris_df.get_column_names(); 

{
    column_names
}

### Drop Species Column

In [ ]:
let numeric_iris_df: DataFrame = iris_df.drop("Species")?;

### Mean

In [ ]:
println!("{}", numeric_iris_df.mean());

### Max

In [ ]:
println!("{}", numeric_iris_df.max());

### Comvert To ndarray

In [ ]:
let numeric_iris_ndarray: ArrayBase<_, _> = numeric_iris_df.to_ndarray::<Float64Type>().unwrap();

In [ ]:
numeric_iris_ndarray

<hr />

## ndarray

### First Element

In [ ]:
numeric_iris_ndarray[[0, 0]]

### First Row

In [ ]:
numeric_iris_ndarray.slice(s![0, ..])

### Shape (rows, cols)

In [ ]:
numeric_iris_ndarray.dim()

### Last Row

In [ ]:
let numeric_iris_ndarray_shape = numeric_iris_ndarray.dim(); // (rows, cols)
// numeric_iris_ndarray.slice(s![numeric_iris_ndarray_shape.1 -1, ..])
// or
numeric_iris_ndarray.slice(s![-1, ..])

### First Five Row

In [ ]:
numeric_iris_ndarray.slice(s![0..5, ..])

### Last Five Row

In [ ]:
numeric_iris_ndarray.slice(s![-5.., ..])

### Check for zero elements

In [ ]:
numeric_iris_ndarray.is_empty()

<hr />

## Mathematics

### Sum

In [ ]:
numeric_iris_ndarray.sum()

### Sum Along Axis

In [ ]:
numeric_iris_ndarray.sum_axis(Axis(0))

### Mean

In [ ]:
numeric_iris_ndarray.mean().unwrap()

### Transpose

In [ ]:
numeric_iris_ndarray.t()

### 2-D matrix multiply

In [ ]:
numeric_iris_ndarray.dot(&numeric_iris_ndarray.t())

<hr />

## Arithmetics

In [ ]:
&numeric_iris_ndarray + 1.0

<hr />

## Sepal Length Column

In [ ]:
numeric_iris_ndarray.column(1)

In [ ]:
let a = numeric_iris_ndarray.column(1);
concatenate(Axis(1), &[numeric_iris_ndarray.column(1), numeric_iris_ndarray.column(1)])

In [ ]:
numeric_iris_df.get_column_names()

## Dataset Dimensions

In [ ]:
// Separate
    let mask = labels.map(|elem| elem == "Iris-virginica");
    let mut count = -1;
    let mut indices = Vec::<usize>::new();
    let mask = labels.map(|elem| {
        count += 1;
        if(elem == "Iris-virginica") { indices.push(count as usize) };
        elem == "Iris-virginica"
        }
    );
    let virginica = features.select(Axis(0), &indices);

<hr />

## Scatter Plot

### Sepal

In [ ]:
let sepal_samples:Vec<(f64,f64)> = {
    let sepal_length_cm: DataFrame = iris_df.select(vec!["SepalLengthCm"]).unwrap();
    let mut sepal_length = sepal_length_cm.to_ndarray::<Float64Type>().unwrap().into_raw_vec().into_iter();
    let sepal_width_cm: DataFrame = iris_df.select(vec!["SepalWidthCm"]).unwrap();
    let mut sepal_width = sepal_width_cm.to_ndarray::<Float64Type>().unwrap().into_raw_vec().into_iter();
    sepal_width.zip(sepal_length).collect()
};

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Iris Dataset", ("Arial", 30).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 3f64..9f64)?;
    
    chart.configure_mesh()
        .x_desc("Sepal Length (cm)")
        .y_desc("Sepal Width (cm)")
        .draw()?;
    
    chart.draw_series(sepal_samples.iter().map(|(x, y)| Circle::new((*x,*y), 3, BLUE.filled())));

    Ok(())
}).style("width:60%")

### Petal

In [ ]:
let petal_samples: Vec<(f64,f64)> = {
    let petal_length_cm: DataFrame = iris_df.select(vec!["PetalLengthCm"]).unwrap();
    let mut petal_length = petal_length_cm.to_ndarray::<Float64Type>().unwrap().into_raw_vec().into_iter();
    let petal_width_cm: DataFrame = iris_df.select(vec!["PetalWidthCm"]).unwrap();
    let mut petal_width = petal_width_cm.to_ndarray::<Float64Type>().unwrap().into_raw_vec().into_iter();
    petal_width.zip(petal_length).collect()
};

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Iris Dataset", ("Arial", 30).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..3f64, 0f64..8f64)?;
    
    chart.configure_mesh()
        .x_desc("Petal Length (cm)")
        .y_desc("Petal Width (cm)")
        .draw()?;
    
    chart.draw_series(petal_samples.iter().map(|(x, y)| Circle::new((*x,*y), 3, GREEN.filled())));

    Ok(())
}).style("width:60%")

## Histogram

### TODO

## Combination of Histogram and Scatter

### TODO

In [ ]:
evcxr_figure((640, 480), |root| {
    let root = root.titled("Scatter with Histogram Example", ("Arial", 20).into_font())?;
    
    let areas = root.split_by_breakpoints([560], [80]);

    let mut x_hist_ctx = ChartBuilder::on(&areas[0])
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 3f64..9f64)?;
    let mut y_hist_ctx = ChartBuilder::on(&areas[3])
        .x_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 3f64..9f64)?;
    let mut scatter_ctx = ChartBuilder::on(&areas[2])
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 3f64..9f64)?;
    scatter_ctx.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;
    scatter_ctx.draw_series(sepal_samples.iter().map(|(x,y)| Circle::new((*x,*y), 3, GREEN.filled())))?;
    let x_hist = Histogram::vertical(&x_hist_ctx)
        .style(RED.filled())
        .margin(0)
        .data(sepal_samples.iter().map(|(x,_)| ((x*10.0) as u32, 0.01)));
    let y_hist = Histogram::horizontal(&y_hist_ctx)
        .style(GREEN.filled())
        .margin(0)
        .data(sepal_samples.iter().map(|(_,y)| ((y*10.0) as u32, 0.01)));
    x_hist_ctx.draw_series(x_hist)?;
    y_hist_ctx.draw_series(y_hist)?;
    
    Ok(())
}).style("width:60%")

## ML Model

In [ ]:
let features: DataFrame = iris_df.select(vec!["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]).unwrap();

In [ ]:
let features: DataFrame = iris_df.select(vec!["SepalLengthCm"]).unwrap();
let array = features.to_ndarray::<Float64Type>().unwrap().into_raw_vec();

In [ ]:
array

In [ ]:
let features: DataFrame = iris_df.select(vec!["SepalWidthCm"]).unwrap();
let array = features.to_ndarray::<Float64Type>().unwrap().into_raw_vec();
array